In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
products = pd.read_csv("https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv", sep=';')
print(products.shape)
products.head()

(200, 5)


,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


In [3]:
restcountries_data = requests.get('https://restcountries.eu/rest/v2/all').json()

countrycode_to_currency = {
    country['alpha2Code']: country['currencies'][0]['code']
    for country in restcountries_data
}
countrycode_to_currency['FR']


'EUR'

In [4]:
def ip_to_countrycode(ip):
    url = f'http://api.ipstack.com/{ip}'
    IPSTACK_KEY = '4630480f447dd17b0ac1f85f6923cb6e'
    data = requests.get(url, params={'access_key': IPSTACK_KEY}).json()
    return data['country_code']

ip_to_countrycode('240.177.79.234')

In [5]:
ip_ok = products.ip_address.str.contains(
    '^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$'
)
codes = [ip_to_countrycode(ip) for ip in products[ip_ok].ip_address]

/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [6]:
products.loc[ip_ok, 'country_code'] = codes
products

,username,ip_address,product,price,infos,country_code
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,NaN
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP
...,...,...,...,...,...,...
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,"Contains gluten, mustard, sugar, milk and fish",CA
196,lprovis5g,191.69.45.257,"Jam - Strawberry, 20 Ml Jar",107.78,Contains gluten and sugar,NaN
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17 RUB,May contain gluten and soja,US
198,mjablonski5i,44.133.211.182,"Mushroom - Chantrelle, Fresh",632.29,Ingredients: soja and sugar,ES


In [7]:
splitted = products['price'].str.split(' ')
price = splitted.str[0].astype(float)
currency = splitted.str[1]

products['price'] = price
products['currency'] = currency

products['currency_guessed'] = products['country_code'].replace(countrycode_to_currency)
products['currency'] = products['currency'].combine_first(products['currency_guessed'])
products = products[products.currency.notna()].reset_index()

In [9]:
currencies_to_eur = requests.get('http://data.fixer.io/api/latest?access_key=53564952f9e96c71b3b710ba0310b8d2').json()['rates']
products['price_eur'] = products.price * products.currency.map(currencies_to_eur)
products

,index,username,ip_address,product,price,infos,country_code,currency,currency_guessed,price_eur
0,4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,JPY,1.137117e+05
1,5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,None,MGA,None,1.425222e+06
2,7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,MXN,2.382106e+03
3,8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,MXN,1.880104e+03
4,10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,"Contains peanut, sugar, egg and fish",US,USD,USD,1.064907e+03
...,...,...,...,...,...,...,...,...,...,...
144,191,ecasserley5b,82.78.162.116,Cup - 8oz Coffee Perforated,714.56,Ingredients: sugar and fish,RO,RON,RON,3.413420e+03
145,193,tliddyard5d,93.22.46.177,Shrimp - Black Tiger 6 - 8,300.03,"Ingredients: sugar, egg and fish",FR,EUR,EUR,3.000300e+02
146,195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.60,"Contains gluten, mustard, sugar, milk and fish",CA,CAD,CAD,1.045986e+03
147,197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17,May contain gluten and soja,US,RUB,USD,3.146444e+04


In [10]:
ing = products.infos\
              .str.replace('^May contain ', 'Contains ')\
              .str.split(' ', 1).str[1]\
              .str.replace(' and ', ', ')\
              .str.get_dummies(sep=', ')

In [11]:
result = pd.concat([products, ing], axis=1)

In [12]:
matrix = products.infos.str.replace('^May contain ', 'Contains ')\
.str.split(' ', 1).str[1].str.replace(' and ', ', ').str.get_dummies(sep=', ')

In [13]:
for col in matrix.columns:
    products[col] = matrix[col]

In [14]:
matrixh = matrix.head(198)
pd.concat([products, matrixh], axis=1)

,index,username,ip_address,product,price,infos,country_code,currency,currency_guessed,price_eur,...,soja,sugar,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,JPY,1.137117e+05,...,0,1,1,1,0,0,0,0,0,1
1,5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,None,MGA,None,1.425222e+06,...,0,1,0,0,0,1,0,0,0,1
2,7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,MXN,2.382106e+03,...,0,1,0,0,0,0,0,0,0,1
3,8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,MXN,1.880104e+03,...,0,1,0,1,0,1,0,0,0,1
4,10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,"Contains peanut, sugar, egg and fish",US,USD,USD,1.064907e+03,...,0,1,1,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,191,ecasserley5b,82.78.162.116,Cup - 8oz Coffee Perforated,714.56,Ingredients: sugar and fish,RO,RON,RON,3.413420e+03,...,0,1,0,1,0,0,0,0,0,1
145,193,tliddyard5d,93.22.46.177,Shrimp - Black Tiger 6 - 8,300.03,"Ingredients: sugar, egg and fish",FR,EUR,EUR,3.000300e+02,...,0,1,1,1,0,0,0,0,0,1
146,195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.60,"Contains gluten, mustard, sugar, milk and fish",CA,CAD,CAD,1.045986e+03,...,0,1,0,1,1,1,1,0,0,1
147,197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17,May contain gluten and soja,US,RUB,USD,3.146444e+04,...,1,0,0,0,1,0,0,0,1,0
